In [4]:
%mkdir -p Sources
!wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/gpu-cpp-tutorial/notebooks/01.02-Execution-Spaces/Sources/ach.h -nv -O Sources/ach.h

2025-03-17 11:36:43 URL:https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/gpu-cpp-tutorial/notebooks/01.02-Execution-Spaces/Sources/ach.h [2893/2893] -> "Sources/ach.h" [1]


In [ ]:
%%writefile Sources/cpu-cooling.cpp

#include <cstdio>
#include <vector>

int main() {
    float k = 0.5;
    float ambient_temp = 20;
    std::vector<float> temp{42,24,50};

    std::printf("step temp[0] temp[1] temp[2] \n");
    for (int step = 0; step < 3; step++) {
        }
        std::printf("%d %.2f    %.2f    %.2f\n", step, temp[0], temp[1], temp[2]);
    }
}

Overwriting Sources/cpu-cooling.cpp


In [13]:
!g++ Sources/cpu-cooling.cpp -o/tmp/a.out

In [14]:
!/tmp/a.out

step temp[0] temp[1] temp[2] 
0 31.00    22.00    35.00
1 25.50    21.00    27.50
2 22.75    20.50    23.75


In [15]:
# We could also have compiled this with the Nvidia Cuda Compiler (NVCC)
!nvcc -x cu Sources/cpu-cooling.cpp -o /tmp/a.out

In [16]:
!/tmp/a.out

step temp[0] temp[1] temp[2] 
0 31.00    22.00    35.00
1 25.50    21.00    27.50
2 22.75    20.50    23.75


In [8]:
%%writefile Sources/gpu-cooling.cpp

#include<algorithm>
#include<cstdio>
#include<vector>

int main() {
    float k = 0.5;
    float ambient_temp = 20;
    std::vector<float> temp {42,24,50};
    auto transformation = [=] (float temp) {return temp + k * (ambient_temp - temp);};

    std::printf("step temp[0] temp[1] temp[2]\n");
    for (int step = 0; step<3; step++){
        std::transform(temp.begin(), temp.end(), temp.begin(), transformation);
        std::printf("%d     %.2f    %.2f    %.2f\n", step, temp[0], temp[1], temp[2]);
    }

}

Overwriting Sources/gpu-cooling.cpp


In [ ]:
!/tmp/a.out

step temp[0] temp[1] temp[2]
0     31.00    22.00    35.00
1     25.50    21.00    27.50
2     22.75    20.50    23.75


In [20]:
# Note: Thrust is a CUDA Core Library which provides standard algorithms and containers that run on the GPU!
#       So here we are using thrust to run functions on the GPU without using CUDA kernels explicitly!

In [2]:
%%writefile Sources/thrust-cooling.cpp

#include <thrust/execution_policy.h>
#include <thrust/universal_vector.h>
#include <thrust/transform.h>
#include <cstdio>

int main() {
    float k = 0.5;
    float ambient_temp = 20;
    thrust::universal_vector<float> temp {42,24,50};
    //This is a way of defining a function which can run on the GPU which is not explicitly a CUDA kernel
    auto transformation = [=] __host__ __device__ (float temp) {return temp + k *(ambient_temp -temp);};

    std::printf("step temp[0] temp[1] temp[2]\n");
    for (int step = 0; step < 3; step++) {
        //This means: call the transform function from the thrust library, make it run from the device, loop from temp.begin() to temp.end() and start 
        //writing to temp.begin() (i.e. overwriting) - the function that youre using is the transformation function (which has been __device__ enabled)
        thrust::transform(thrust::device, temp.begin(), temp.end(), temp.begin(), transformation);
        std::printf("%d     %.2f    %.2f    %.2f\n", step, temp[0], temp[1], temp[2]);
    }
}

Overwriting Sources/thrust-cooling.cpp


In [17]:
!nvcc --extended-lambda Sources/thrust-cooling.cpp -x cu -arch=native -o /tmp/a.out

In [19]:
!/tmp/a.out

step temp[0] temp[1] temp[2]
0     31.00    22.00    35.00
1     25.50    21.00    27.50
2     22.75    20.50    23.75
